# 🗣️ Unit 4.1: NLP Fundamentals

**Course:** Advanced Machine Learning (AICC 303)  
**Topics:**
*   4.2 Text Tokenization and Normalization
*   4.3 Lemmatization and Stemming
*   4.4 Text Embedding (Bag of Words, TF-IDF, Word2Vec)

**Goal:** Computers don't understand text. We must convert text into numbers (vectors) before feeding it to a model.

---

In [5]:
!pip install nltk


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fulbutte\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\fulbutte\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fulbutte\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fulbutte\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\fulbutte\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## 1. Text Preprocessing

Raw text is messy. We need to clean it.
1.  **Lowercasing**: "Apple" == "apple"
2.  **Tokenization**: Splitting sentences into words.
3.  **Stopword Removal**: Removing common words like "the", "is", "and".
4.  **Stemming/Lemmatization**: Converting "running" -> "run".

In [7]:
text = "The quick brown foxes are jumping over the lazy dog. It was amazing!"

# 1. Tokenization
tokens = nltk.word_tokenize(text.lower())
print(f"Tokens: {tokens}")

# 2. Stopword Removal
stop_words = set(stopwords.words('english'))
filtered_tokens = [w for w in tokens if w.isalnum() and w not in stop_words]
print(f"Filtered: {filtered_tokens}")

# 3. Stemming vs Lemmatization
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

stemmed = [stemmer.stem(w) for w in filtered_tokens]
lemmatized = [lemmatizer.lemmatize(w) for w in filtered_tokens]

print(f"Stemmed: {stemmed}")
print(f"Lemmatized: {lemmatized}")

# Note: Stemmers chop off ends (foxes->fox), Lemmatizers use dictionary (better but slower).

Tokens: ['the', 'quick', 'brown', 'foxes', 'are', 'jumping', 'over', 'the', 'lazy', 'dog', '.', 'it', 'was', 'amazing', '!']
Filtered: ['quick', 'brown', 'foxes', 'jumping', 'lazy', 'dog', 'amazing']
Stemmed: ['quick', 'brown', 'fox', 'jump', 'lazi', 'dog', 'amaz']
Lemmatized: ['quick', 'brown', 'fox', 'jumping', 'lazy', 'dog', 'amazing']


## 2. Text Representation (Vectorization)

### 2.1 Bag of Words (BoW)
Creates a matrix where rows are documents and columns are unique words. Value = Count.

In [8]:
corpus = [
    "I love machine learning.",
    "Machine learning is great.",
    "I love deep learning."
]

vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(corpus)

print("Vocabulary:", vectorizer.get_feature_names_out())
print("BoW Matrix:\n", X_bow.toarray())

Vocabulary: ['deep' 'great' 'is' 'learning' 'love' 'machine']
BoW Matrix:
 [[0 0 0 1 1 1]
 [0 1 1 1 0 1]
 [1 0 0 1 1 0]]


### 2.2 TF-IDF (Term Frequency - Inverse Document Frequency)
Instead of counts, we weight words.
*   **TF**: High if word appears often in THIS document.
*   **IDF**: Low if word appears in MANY documents (like "the").

Helps find "unique" keywords.

In [9]:
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(corpus)

print("TF-IDF Matrix:\n", np.round(X_tfidf.toarray(), 2))

TF-IDF Matrix:
 [[0.   0.   0.   0.48 0.62 0.62]
 [0.   0.58 0.58 0.35 0.   0.44]
 [0.72 0.   0.   0.43 0.55 0.  ]]


## 3. Word Embeddings (Deep Learning)

BoW and TF-IDF are "Sparse" and high-dimensional. They don't capture meaning (e.g., "King" and "Queen" are unrelated).

**Embeddings** map words to dense vectors (e.g., size 50 or 100) where similar words are close in space.

### Word2Vec / GloVe
Algorithms trained on massive text (Wikipedia) to learn these relationships.
*   King - Man + Woman ≈ Queen

In [10]:
import torch
import torch.nn as nn

# PyTorch Embedding Layer Example
vocab_size = 50
embedding_dim = 8

# Simple tokenization (word to index)
word_to_idx = {}
for doc in corpus:
    for word in doc.lower().split():
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)

# Create embedding layer
embedding = nn.Embedding(len(word_to_idx), embedding_dim)

# Example: get embedding for a word
word = "machine"
if word in word_to_idx:
    word_idx = torch.LongTensor([word_to_idx[word]])
    word_embedding = embedding(word_idx)
    print(f"\nWord: '{word}'")
    print(f"Embedding (Vector of size {embedding_dim}):\n{word_embedding.detach().numpy()}")


Word: 'machine'
Embedding (Vector of size 8):
[[-0.25702578  0.6040811   0.95967674  2.116546   -0.09270746 -0.6710528
   1.510489    1.5074047 ]]
